## Connected components

For all interactomes:
* Number of connected components
* Size of the largest connected component
* Relative size of the largest connected component
* Average size of the connected components
* Size of the smallest component
* Number of isolated nodes

For all pathways:
- Sizes of connected components for all pathways

In [1]:
import bisect

import networkx as nx
import pandas as pd
import seaborn
import seaborn as sns

import config
from config import GRAPHS_PATH
from lib.networks import get_interactomes, get_multiindex
from lib.networks import create_pathway_interaction_networks
from visualization.visualize_single_network import plot_pathway_all_levels, Coloring

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

config.set_root_wd()


Working directory: c:\git\pathwayanalysisplatform\proteoformnetworks


In [2]:
index = get_multiindex()
interactomes_no_sm, interactomes_with_sm, interactomes_with_unique_sm = get_interactomes(GRAPHS_PATH)
interactomes = [*interactomes_no_sm.values(), *interactomes_with_sm.values(), *interactomes_with_unique_sm.values()]

In [3]:
from lib.networks import get_multiindex
import itertools


def getTuples(i):
    sizes = [len(c) / i.number_of_nodes() for c in sorted(nx.connected_components(i), key=len, reverse=True)]
    d = [(i.graph["method"], i.graph["level"], s) for s in sizes]
    return d


tuples = [getTuples(i) for i in interactomes]
data = list(itertools.chain.from_iterable(tuples))
data
df_cc = pd.DataFrame(data, columns=['Small Molecules', 'Entity Level', 'Size'])
df_cc

,Small Molecules,Entity Level,Size
0,no_sm,genes,0.81
1,no_sm,genes,0.01
2,no_sm,genes,0.00
3,no_sm,genes,0.00
4,no_sm,genes,0.00
...,...,...,...
13277,with_unique_sm,proteoforms,0.00
13278,with_unique_sm,proteoforms,0.00
13279,with_unique_sm,proteoforms,0.00
13280,with_unique_sm,proteoforms,0.00


In [4]:
# Number of connected components
# Size of the largest connected component
# Relative size of the largest connected component
# Average size of the connected components
# Size of the smallest component
# Number of isolated nodes

def getTuples(i):
    ccs_sizes = [len(c) for c in sorted(nx.connected_components(i), key=len, reverse=False)]
    num_ccs = len(ccs_sizes)

    return (
        i.graph["method"],
        i.graph["level"],
        num_ccs,
        ccs_sizes[-1],
        ccs_sizes[-1] / i.number_of_nodes(),
        sum(ccs_sizes) / len(ccs_sizes),
        ccs_sizes[0],
        0 if ccs_sizes[0] > 1 else bisect.bisect_left(ccs_sizes, 2, lo=0, hi=len(ccs_sizes))
    )


data = [getTuples(i) for i in interactomes]
columns = [
    'Method',
    'Level',
    'Number of connected components',
    'Size of largest connected component',
    'Relative size of largest connected component',
    'Average size of connected components',
    "Size of smallest connected component",
    "Number of isolated nodes"
]
df_interactomes = pd.DataFrame.from_records(data, columns=columns, index=get_multiindex())
df_interactomes[columns[2:]]

Number of connected components  \
Small Molecules          Entity Level                                   
Not Included             genes                                   1773   
                         proteins                                1774   
                         proteoforms                             1821   
Included                 genes                                    195   
                         proteins                                 195   
                         proteoforms                              205   
Reaction-Unique Included genes                                   2427   
                         proteins                                2427   
                         proteoforms                             2465   

                                       Size of largest connected component  \
Small Molecules          Entity Level                                        
Not Included             genes                                        8869   
                         proteins                                     8966   
                         proteoforms                                 11984   
Included                 genes                                       12692   
                         proteins                                    12790   
                         proteoforms                                 15962   
Reaction-Unique Included genes                                       28696   
                         proteins                                    28793   
                         proteoforms                                 31644   

                                       Relative size of largest connected component  \
Small Molecules          Entity Level                                                 
Not Included             genes                                                 0.81   
                         proteins                                              0.81   
                         proteoforms                                           0.84   
Included                 genes                                                 0.98   
                         proteins                                              0.98   
                         proteoforms                                           0.98   
Reaction-Unique Included genes                                                 0.73   
                         proteins                                              0.73   
                         proteoforms                                           0.75   

                                       Average size of connected components  \
Small Molecules          Entity Level                                         
Not Included             genes                                         6.19   
                         proteins                                      6.24   
                         proteoforms                                   7.85   
Included                 genes                                        66.32   
                         proteins                                     66.83   
                         proteoforms                                  79.32   
Reaction-Unique Included genes                                        16.10   
                         proteins                                     16.14   
                         proteoforms                                  17.20   

                                       Size of smallest connected component  \
Small Molecules          Entity Level                                         
Not Included             genes                                            1   
                         proteins                                         1   
                         proteoforms                                      1   
Included                 genes                                            1   
                         proteins                                         

In [5]:
from pathlib import Path
from config import METHODS, PATHWAY_GRAPHS_PATH
from config import LEVELS
from lib.networks import get_json_filename, create_pathway_interaction_network, read_graph, replace_methods
from lib.graph_database_access import get_pathways

columns_for_pathways = [
    'Pathway',
    'Small Molecules',
    'Entity Level',
    'Relative size of largest connected component',
    'Average relative size of connected components',
    'Relative Size of smallest connected component',
    'Relative Number of isolated nodes',
    'Number of connected components'
]


def get_pathway_tuples(pathway, pathway_graph):
    ccs_sizes = [len(cc) for cc in sorted(nx.connected_components(pathway_graph), key=len, reverse=False)]
    rel_ccs_sizes = [size / pathway_graph.number_of_nodes() for size in ccs_sizes]

    return (
        pathway,
        pathway_graph.graph["method"],
        pathway_graph.graph["level"],
        rel_ccs_sizes[-1],
        sum(rel_ccs_sizes) / len(rel_ccs_sizes),
        rel_ccs_sizes[0],
        0 if ccs_sizes[0] > 1 else bisect.bisect_left(ccs_sizes, 2, lo=0, hi=len(ccs_sizes)) / len(ccs_sizes),
        len(ccs_sizes)
    )


data_pathways = []
pathways = get_pathways()["stId"]
for pathway in pathways:
    for method in METHODS:
        for level in LEVELS:
            filename = get_json_filename(level, method, PATHWAY_GRAPHS_PATH, pathway)
            if not Path(filename).exists():
                create_pathway_interaction_network(pathway, level, method, PATHWAY_GRAPHS_PATH)
            G = read_graph(filename)
            if G.number_of_nodes() > 0:
                data_pathways.append(get_pathway_tuples(pathway, G))

df_pathways = pd.DataFrame.from_records(data_pathways, columns=columns_for_pathways)
replace_methods(df_pathways)
df_pathways

,Pathway,Small Molecules,Entity Level,Relative size of largest connected component,Average relative size of connected components,Relative Size of smallest connected component,Relative Number of isolated nodes,Number of connected components
0,R-HSA-9613829,Not Included,genes,1.00,1.00,1.00,0.00,1
1,R-HSA-9613829,Not Included,proteins,1.00,1.00,1.00,0.00,1
2,R-HSA-9613829,Not Included,proteoforms,1.00,1.00,1.00,0.00,1
3,R-HSA-9613829,Included,genes,1.00,1.00,1.00,0.00,1
4,R-HSA-9613829,Included,proteins,1.00,1.00,1.00,0.00,1
...,...,...,...,...,...,...,...,...
18403,R-HSA-917729,Included,proteins,1.00,1.00,1.00,0.00,1
18404,R-HSA-917729,Included,proteoforms,1.00,1.00,1.00,0.00,1
18405,R-HSA-917729,Reaction-Unique Included,genes,1.00,1.00,1.00,0.00,1
18406,R-HSA-917729,Reaction-Unique Included,proteins,1.00,1.00,1.00,0.00,1


In [11]:
# - Select records at protein level
# - Select records with and without small molecules
df_no_sm = df_pathways[(df_pathways['Small Molecules'] == "Not Included") & (df_pathways['Entity Level'] == f"{config.proteins}")]
df_with_sm = df_pathways[(df_pathways['Small Molecules'] == "Included") & (df_pathways['Entity Level'] == f"{config.proteins}")]
df_with_ru_sm =  df_pathways[(df_pathways['Small Molecules'] == "Reaction-Unique Included") & (df_pathways['Entity Level'] == f"{config.proteins}")]

In [20]:
df_no_sm_size_lcc = df_no_sm[['Pathway', 'Relative size of largest connected component']]
df_no_sm_size_lcc.set_index('Pathway', inplace=True)

df_with_sm_size_lcc = df_with_sm[['Pathway', 'Relative size of largest connected component']]
df_with_sm_size_lcc.set_index('Pathway', inplace=True)

result = pd.concat([df_no_sm_size_lcc, df_with_sm_size_lcc], axis=1)
result = result[~result.isnull().any(axis=1)]
result.columns = ["LCC size no sm", "LCC size with sm"]
result["Increase"] = result["LCC size with sm"]/result["LCC size no sm"]
result.sort_values(by=['Increase'], ascending=False, inplace=True)
result.head(10)

,LCC size no sm,LCC size with sm,Increase
Pathway,,,
R-HSA-70263,0.03,1.00,34.00
R-HSA-1482839,0.03,1.00,29.00
R-HSA-1482788,0.04,1.00,27.00
R-HSA-1855204,0.04,1.00,26.00
R-HSA-75876,0.04,1.00,24.00
R-HSA-3295583,0.04,0.89,22.22
R-HSA-1482801,0.05,1.00,22.00
R-HSA-379726,0.05,1.00,21.00
R-HSA-189200,0.05,1.00,21.00


In [28]:
df_no_sm_num_ccs = df_no_sm[['Pathway', 'Number of connected components']]
df_no_sm_num_ccs.set_index('Pathway', inplace=True)

df_with_sm_num_ccs = df_with_sm[['Pathway', 'Number of connected components']]
df_with_sm_num_ccs.set_index('Pathway', inplace=True)

result = pd.concat([df_no_sm_num_ccs, df_with_sm_num_ccs], axis=1)
result = result[~result.isnull().any(axis=1)]
result.columns = ["Num CCs no sm", "Num CCs with sm"]
result["Increase"] = result["Num CCs with sm"]/result["Num CCs no sm"]
result.sort_values(by=['Increase'], ascending=True, inplace=True)
result.head(10)

,Num CCs no sm,Num CCs with sm,Increase
Pathway,,,
R-HSA-6811558,82.00,1,0.01
R-HSA-109704,41.00,1,0.02
R-HSA-1660499,40.00,1,0.03
R-HSA-1483166,38.00,1,0.03
R-HSA-1660661,37.00,1,0.03
R-HSA-70263,34.00,1,0.03
R-HSA-4085001,30.00,1,0.03
R-HSA-1482839,29.00,1,0.03
R-HSA-1482788,27.00,1,0.04


In [33]:
df_with_ru_sm_num_ccs = df_with_ru_sm[['Pathway', 'Number of connected components']]
df_with_ru_sm_num_ccs.set_index('Pathway', inplace=True)

result = pd.concat([df_with_sm_num_ccs, df_with_ru_sm_num_ccs], axis=1)
result = result[~result.isnull().any(axis=1)]
result.columns = ["Num CCs with sm", "Num CCs with reaction-unique sm"]
result["Increase"] = result["Num CCs with reaction-unique sm"]/result["Num CCs with sm"]
result.sort_values(by=['Increase'], ascending=False, inplace=True)
result.sample(10)

,Num CCs with sm,Num CCs with reaction-unique sm,Increase
Pathway,,,
R-HSA-75955,1,1,1.00
R-HSA-8941856,1,1,1.00
R-HSA-9609523,1,1,1.00
R-HSA-376172,1,1,1.00
R-HSA-77350,1,4,4.00
R-HSA-975871,8,8,1.00
R-HSA-2470946,1,1,1.00
R-HSA-3214841,1,2,2.00
R-HSA-111997,4,5,1.25


In [ ]:
# df_no_sm_size_lcc = df_no_sm[['Pathway', 'Average relative size of connected components']]
# df_no_sm_size_lcc.set_index('Pathway', inplace=True)
# df_with_sm_size_cc = df_with_sm[['Pathway', 'Average relative size of connected components']]
# df_with_sm_size_cc.set_index('Pathway', inplace=True)


In [ ]:
df_pathways_grouped = df_pathways.groupby(['Small Molecules', 'Entity Level']).agg(
    {
        'Relative size of largest connected component': ['mean'],
        'Average relative size of connected components': ['mean'],
        'Relative Size of smallest connected component': ['mean'],
        'Relative Number of isolated nodes': ['mean']
    })
df_pathways_grouped.columns = [
    'Avg. Rel. Size of LCC',
    'Avg. Rel. Size of CCs',
    'Avg. Rel. Size of SCC',
    'Avg. Rel. Number of isolated nodes'
]
df_pathways_grouped = df_pathways_grouped.set_index(get_multiindex())

df_pathways_grouped

In [ ]:
from config import SMALL_MOLECULES

g = sns.catplot(x="Entity Level", y='Relative size of largest connected component',
                col="Small Molecules",
                data=df_pathways.sample(1000),
                kind="violin", split=True,
                order=LEVELS, col_order=SMALL_MOLECULES,
                height=4, aspect=.7);
# g.fig.suptitle("Small Molecules")
# g.fig.subplots_adjust(top=0.85)
g.set_titles(" Small molecules\n{col_name}")
g.set_axis_labels("Level", 'Rel.Size of LCC')
g.add_legend()

In [ ]:
g = sns.catplot(x="Entity Level", y='Relative size of largest connected component',
                col="Small Molecules",
                data=df_pathways.sample(1000),
                kind="strip", split=True,
                order=LEVELS, col_order=SMALL_MOLECULES,
                height=4, aspect=.7);
# g.fig.suptitle("Small Molecules")
# g.fig.subplots_adjust(top=0.85)
g.set_titles(" Small molecules\n{col_name}")
g.set_axis_labels("Level", 'Rel.Size of LCC')
g.add_legend()

In [ ]:
from lib.networks import get_pathway_filenames

n = 10
pathway_examples_isolated_nodes = set()
for pathway, filename in get_pathway_filenames(PATHWAY_GRAPHS_PATH):
    G = read_graph(filename)
    if G.number_of_nodes() < 20:
        ccs_sizes = [len(cc) for cc in sorted(nx.connected_components(G), key=len, reverse=False)]
        if ccs_sizes[0] == 1:
            prev = len(pathway_examples_isolated_nodes)
            pathway_examples_isolated_nodes.add(pathway)
            if len(pathway_examples_isolated_nodes) != prev:
                print(f"{pathway} CCs: {ccs_sizes}")
            if len(pathway_examples_isolated_nodes) == n: break;


In [ ]:
for pathway in pathway_examples_isolated_nodes:
    graphs = create_pathway_interaction_networks(pathway, PATHWAY_GRAPHS_PATH)
    p = plot_pathway_all_levels(pathway, out_path=PATHWAY_GRAPHS_PATH, graphs=graphs,
                                coloring=Coloring.ENTITY_TYPE,
                                highlight_articulations=False, highlight_bridges=False,
                                node_size = 10,
                                toolbar_location=None)